In [37]:
from pyomo.environ import *
import pandas as pd
import numpy as np
from data_processing import load_all_data

#IMPORTAÇÃO DE DADOS:

In [84]:
data = load_all_data()
D_GEN = data['D_GEN']
D_LIN = data['D_LIN']
D_LOAD = data['D_LOAD']


In [85]:
#Deixando os dados em horas e nos formatos conhecidos
D_GEN['Lambda'] = D_GEN['Falhas/ano'] / 8760 # 8760 horas em um ano
D_GEN['MTTF'] = 1/D_GEN['Lambda'] #Tempo médio entre falhas
D_GEN['mu'] = 1/D_GEN['Tempo de reparo(h)'] #Taxa de reparo/hora
D_GEN['A'] = D_GEN['mu']/(D_GEN['mu'] + D_GEN['Lambda']) #Disponibilidade
D_GEN['U'] = 1 - D_GEN['A'] #Indisponibilidade

In [86]:
Ng = len(D_GEN)
geradores = np.arange(1,Ng+1)
Nl = len(D_LIN)
linhas = np.arange(1,Nl+1)
Nc = len(D_LOAD)
cargas = np.arange(1,Nc+1)
print(cargas)

[1 2 3 4 5 6]


In [111]:
#Criando modelo Concreto:
model = ConcreteModel()

In [112]:
#Criando Sets do Problema:
model.ger = RangeSet(Ng) #Geradores
model.line = RangeSet(Nl) #Linhas
model.carga = RangeSet(Nc) #Cargas
model.bus = RangeSet(6) #Barras

In [113]:
#Parâmtro das Cargas:
model.p_carga = Param(model.carga,initialize={cargas[i]: D_LOAD.loc[i, 'Carga'] for i in range(len(cargas))})
model.carga_bus = Param(model.carga,initialize={cargas[i]: D_LOAD.loc[i, 'Barra'] for i in range(len(cargas))})

#Parâmtros das Linhas:
model.barra_de = Param(model.line,initialize={linhas[i]: D_LIN.loc[i, 'Barra De'] for i in range(len(linhas))})
model.barra_para = Param(model.line,initialize={linhas[i]: D_LIN.loc[i, 'Barra Para'] for i in range(len(linhas))})
model.x = Param(model.line,initialize={linhas[i]: D_LIN.loc[i, 'Reatância'] for i in range(len(linhas))})
model.flux_max = Param(model.line,initialize={linhas[i]: D_LIN.loc[i, 'Capacidade'] for i in range(len(linhas))})

#Parâmetros dos Geradores:
model.ger_bus = Param(model.ger,initialize={geradores[i]: D_GEN.loc[i, 'Barra'] for i in range(len(geradores))})

In [120]:
#Variáveis:
model.pg = Var(model.ger, domain=NonNegativeReals)
model.pd = Var(model.carga, domain=NonNegativeReals)
model.theta = Var(model.bus, bounds=(-np.pi, np.pi), domain=Reals)

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually indicative of
a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [ ]:
#Restrições:
#Balanço de Potência nas Barras
def balance_power_rule(model, barra):
    PG = sum(model.pg[ger] for ger in model.ger if model.ger_bus[ger] == barra)
    PL = sum(model.p_carga[car] for car in model.carga if model.carga_bus[car] == barra)
    fluxo = 0
    for line in model.line:
        if model.barra_de[line] == barra:
            fluxo += (model.theta[barra] - model.theta[model.barra_para[line]]) / model.x[line]
        elif model.barra_para[line] == barra:
            fluxo += (model.theta[barra] - model.theta[model.barra_de[line]]) / model.x[line]
    PD = sum(model.pd[carga] for carga in model.carga if model.carga_bus[carga] == barra)
    return PG + PD + fluxo - PL == 0
model.balance_power = Constraint(model.bus, rule=balance_power_rule)

In [117]:
#Limites Térmicos das Linhas:
def flow_limit_rule(model, line):
    bus1 = model.barra_de[line]
    bus2 = model.barra_para[line]
    flux = (model.theta[bus1] - model.theta[bus2]) / model.x[line]
    return flux <= model.flux_max[line]
model.flow_limit = Constraint(model.line, rule=flow_limit_rule)


In [119]:
#Limita o corte ao valor da carga:
def load_shed_limit_rule(model, carga):
    return model.pd[carga] <= model.p_carga[carga]
model.shed_limit = Constraint(model.carga, rule=load_shed_limit_rule)

In [121]:
#Função Objetivo: Menor perda de carga possível:
def objective_function(model):
    PD_sum = sum(model.pd[carga] for carga in model.carga)
    return PD_sum
model.objective = Objective(rule=objective_function, sense=minimize)